In [1]:
import pandas as pd
import numpy as np
from importlib import reload
%load_ext line_profiler
pd.options.display.max_columns = 100

## Download data

In [2]:
import sys
import os

sys.path.append(os.path.abspath('../..'))

In [3]:
data_root = os.path.abspath('../../_data/kaggle/rossman')
#!kaggle competitions download -c rossmann-store-sales -p $data_root

In [6]:
import urllib.request
import os

os.makedirs(data_root, exist_ok=True)
urllib.request.urlretrieve('http://files.fast.ai/part2/lesson14/rossmann.tgz', os.path.join(data_root, 'rossmann.tgz'))

('D:\\learn\\online_course\\notes\\_data\\kaggle\\rossman\\rossmann.tgz',
 <http.client.HTTPMessage at 0x16c094219b0>)

In [4]:
from glob import glob
import shutil
#for zipfile in glob(data_root + '/*.(zip)|(tgz)'):
for zipfile in glob(data_root + '/*.tgz'):
    shutil.unpack_archive(zipfile, data_root)

In [5]:
ls $data_root

 Volume in drive D is Work
 Volume Serial Number is 08AF-55F7

 Directory of D:\learn\online_course\notes\_data\kaggle\rossman

07/11/2018  12:48 PM    <DIR>          .
07/11/2018  12:48 PM    <DIR>          ..
01/11/2017  12:16 PM            86,605 googletrend.csv
07/13/2018  02:24 PM         7,730,448 rossmann.tgz
09/30/2015  06:48 AM           317,611 sample_submission.csv
01/11/2017  09:19 AM               265 state_names.csv
09/30/2015  06:48 AM            45,010 store.csv
01/07/2017  06:17 AM             9,051 store_states.csv
09/30/2015  06:48 AM         1,427,425 test.csv
09/30/2015  06:48 AM        38,057,952 train.csv
01/11/2017  12:13 PM         1,518,814 weather.csv
               9 File(s)     49,193,181 bytes
               2 Dir(s)  184,084,017,152 bytes free


## Preprocessing

In [6]:
import pandas as pd
from pandas_summary import DataFrameSummary
from IPython.display import display
from glob import glob

In [7]:
table_names = [os.path.splitext(os.path.basename(f))[0] for f in glob(data_root + '/*.csv')]
tables = [pd.read_csv(f, low_memory=False) for f in glob(data_root + '/*.csv')]
print(', '.join(table_names))

googletrend, sample_submission, state_names, store, store_states, test, train, weather


In [8]:
for name, tb in zip(table_names, tables):
    print(name)
    display(tb.head())

googletrend


,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67


sample_submission


,Id,Sales
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


state_names


...

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


train


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


weather


,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_PressurehPa,Mean_Sea_Level_PressurehPa,Min_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,65,1013,1005,1001,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,78,1028,1022,1014,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,77,1035,1030,1026,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,87,1036,1035,1034,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,87,1035,1034,1033,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268


In [9]:
for tb in tables:
    display(DataFrameSummary(tb).summary())

D:\dev\libs\Anaconda3\envs\tf140p35\lib\site-packages\pandas_summary\__init__.py:86: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  stats = pd.concat([counts, uniques, missing], axis=1)
D:\dev\libs\Anaconda3\envs\tf140p35\lib\site-packages\pandas_summary\__init__.py:63: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([self.df.describe(), self.columns_stats])[self.df.columns]


,file,week,trend
count,NaN,NaN,2072
mean,NaN,NaN,63.8142
std,NaN,NaN,12.6502
min,NaN,NaN,0
25%,NaN,NaN,55
50%,NaN,NaN,64
75%,NaN,NaN,72
max,NaN,NaN,100
counts,2072,2072,2072
uniques,14,148,68


,Id,Sales
count,41088,41088
mean,20544.5,0
std,11861.2,0
min,1,0
25%,10272.8,0
50%,20544.5,0
75%,30816.2,0
max,41088,0
counts,41088,41088
uniques,41088,1


,StateName,State
count,16,16
unique,16,16
top,Niedersachsen,MV
freq,1,1
counts,16,16
uniques,16,16
missing,0,0
missing_perc,0%,0%
types,unique,unique


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
count,1115,NaN,NaN,1112,761,761,1115,571,571,NaN
mean,558,NaN,NaN,5404.9,7.2247,2008.67,0.512108,23.5954,2011.76,NaN
std,322.017,NaN,NaN,7663.17,3.21235,6.19598,0.500078,14.142,1.67494,NaN
min,1,NaN,NaN,20,1,1900,0,1,2009,NaN
25%,279.5,NaN,NaN,717.5,4,2006,0,13,2011,NaN
50%,558,NaN,NaN,2325,8,2010,1,22,2012,NaN
75%,836.5,NaN,NaN,6882.5,10,2013,1,37,2013,NaN
max,1115,NaN,NaN,75860,12,2015,1,50,2015,NaN
counts,1115,1115,1115,1112,761,761,1115,571,571,571
uniques,1115,4,3,654,12,23,2,24,7,3


,Store,State
count,1115,NaN
mean,558,NaN
std,322.017,NaN
min,1,NaN
25%,279.5,NaN
50%,558,NaN
75%,836.5,NaN
max,1115,NaN
counts,1115,1115
uniques,1115,12


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
count,41088,41088,41088,NaN,41077,41088,NaN,41088
mean,20544.5,555.9,3.97917,NaN,0.854322,0.395833,NaN,0.443487
std,11861.2,320.274,2.01548,NaN,0.352787,0.489035,NaN,0.496802
min,1,1,1,NaN,0,0,NaN,0
25%,10272.8,279.75,2,NaN,1,0,NaN,0
50%,20544.5,553.5,4,NaN,1,0,NaN,0
75%,30816.2,832.25,6,NaN,1,1,NaN,1
max,41088,1115,7,NaN,1,1,NaN,1
counts,41088,41088,41088,41088,41077,41088,41088,41088
uniques,41088,856,7,48,2,2,2,2


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
count,1.01721e+06,1.01721e+06,NaN,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,NaN,1.01721e+06
mean,558.43,3.99834,NaN,5773.82,633.146,0.830107,0.381515,NaN,0.178647
std,321.909,1.99739,NaN,3849.93,464.412,0.375539,0.485759,NaN,0.383056
min,1,1,NaN,0,0,0,0,NaN,0
25%,280,2,NaN,3727,405,1,0,NaN,0
50%,558,4,NaN,5744,609,1,0,NaN,0
75%,838,6,NaN,7856,837,1,1,NaN,0
max,1115,7,NaN,41551,7388,1,1,NaN,1
counts,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209
uniques,1115,7,942,21734,4086,2,2,4,2


,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_PressurehPa,Mean_Sea_Level_PressurehPa,Min_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
count,NaN,NaN,15840,15840,15840,15840,15840,15840,15840,15840,15840,15840,15840,15840,15459,15459,15459,15840,15840,3604,15840,14667,NaN,15840
mean,NaN,NaN,14.6441,10.389,6.19899,8.58782,6.20581,3.62614,93.6596,74.2829,50.1586,1018.53,1015.3,1012.31,24.0576,12.2398,7.02516,22.7666,11.9722,48.8643,0.831718,5.55131,NaN,175.897
std,NaN,NaN,8.64601,7.37926,6.52639,6.24478,6.08677,6.12839,7.67853,13.4866,19.9602,7.78872,8.12331,8.60058,8.9768,5.06794,4.9806,8.98862,5.87284,13.027,2.51351,1.68771,NaN,101.589
min,NaN,NaN,-11,-13,-15,-14,-15,-73,44,30,4,976,974,970,0,0,0,3,2,21,0,0,NaN,-1
25%,NaN,NaN,8,4,1,4,2,-1,90.75,65,34,1014,1010,1007,14,10,3,16,8,39,0,5,NaN,80
50%,NaN,NaN,15,11,7,9,7,4,94,76,49,1019,1016,1013,31,11,7,21,11,48,0,6,NaN,202
75%,NaN,NaN,21,16,11,13,11,8,100,85,66,1024,1021,1018,31,14,10,27,14,55,0.25,7,NaN,256
max,NaN,NaN,39,31,24,25,20,19,100,100,100,1043,1040,1038,31,31,31,101,53,111,58.93,8,NaN,360
counts,15840,15840,15840,15840,15840,15840,15840,15840,15840,15840,15840,15840,15840,15840,15459,15459,15459,15840,15840,3604,15840,14667,11889,15840
uniques,16,990,51,45,40,40,36,40,53,71,93,67,67,69,24,32,24,44,29,47,41,9,21,362


In [10]:
googletrend, sample_submission, state_names, store, store_states, test, train, weather = tables

In [11]:
len(test), len(train)

(41088, 1017209)

In [12]:
sorted(train.Date.unique())[-10:]

['2015-07-22',
 '2015-07-23',
 '2015-07-24',
 '2015-07-25',
 '2015-07-26',
 '2015-07-27',
 '2015-07-28',
 '2015-07-29',
 '2015-07-30',
 '2015-07-31']

In [13]:
sorted(test.Date.unique())[:10]

['2015-08-01',
 '2015-08-02',
 '2015-08-03',
 '2015-08-04',
 '2015-08-05',
 '2015-08-06',
 '2015-08-07',
 '2015-08-08',
 '2015-08-09',
 '2015-08-10']

In [14]:
googletrend.file.unique()

array(['Rossmann_DE_SN', 'Rossmann_DE', 'Rossmann_DE_BY',
       'Rossmann_DE_TH', 'Rossmann_DE_NW', 'Rossmann_DE_BE',
       'Rossmann_DE_RP', 'Rossmann_DE_BW', 'Rossmann_DE_NI',
       'Rossmann_DE_SH', 'Rossmann_DE_HE', 'Rossmann_DE_ST',
       'Rossmann_DE_HH', 'Rossmann_DE_SL'], dtype=object)

In [15]:
# clean up weath
weather['Date'] = pd.to_datetime(weather.Date)

In [16]:
# clean up trend
clean_trend = googletrend.copy()
clean_trend['Date'] = pd.to_datetime(googletrend.week.str.split(' - ', expand=True)[0])
clean_trend['State'] = googletrend.file.str.split('_', expand=True)[2]
de_trend = clean_trend[clean_trend.file == 'Rossmann_DE']
clean_trend = clean_trend.drop(columns=['week', 'file'])
de_trend = de_trend.drop(columns=['week', 'file', 'State'])
clean_trend.head()

,trend,Date,State
0,96,2012-12-02,SN
1,95,2012-12-09,SN
2,91,2012-12-16,SN
3,48,2012-12-23,SN
4,67,2012-12-30,SN


In [17]:
de_trend.head()

,trend,Date
148,77,2012-12-02
149,76,2012-12-09
150,85,2012-12-16
151,59,2012-12-23
152,61,2012-12-30


In [18]:
from datetime import datetime

def expend_date(df, expend=None):
    df = df.copy()
    df['Date'] = pd.to_datetime(df.Date)
    
    if not expend:
        expend = ['Year', 'Month', 'Day', 'Week']
    d = {'Year': df.Date.dt.year,
         'Month': df.Date.dt.month,
         'Day': df.Date.dt.day,
         'Week': df.Date.dt.week}
    for label in expend:
        df[label] = d[label]

    return df
    
def preprocess(data_set, joins, drops=[], nans=[]):
    d = expend_date(data_set)
    
    for tmp in joins:
        if len(tmp) == 3:
            right, left_on, right_on = tmp
        else:
            right, left_on = tmp
            right_on = left_on
        d = pd.merge(d, right, how='left', left_on=left_on, right_on=right_on, suffixes=("", "_r"))
    if drops:
        d = d.drop(columns=drops)
    
    for tmp in nans:
        if len(tmp) == 2:
            name, val = tmp
            d[name] = d[name].fillna(val)
        else:
            name, val, val_type = tmp
            d[name] = d[name].fillna(val).astype(val_type)
    # StatHolidy from ['0', 'a', 'b', 'c'] to [True, False]
    d.StateHoliday = data_set.StateHoliday == '0'
    
    # create new feature 'CompetitionOpenSince'
#     d['CompetitionOpenSince'] = pd.to_datetime(
#         d.apply(lambda x: datetime(x.CompetitionOpenSinceYear, x.CompetitionOpenSinceMonth, 15), axis=1))
    d['CompetitionOpenSince'] = pd.to_datetime(
                                    (d.CompetitionOpenSinceYear * 10000 + d.CompetitionOpenSinceMonth * 100 + 15).apply(str),
                                    format="%Y%m%d")
    d['CompetitionDaysOpen'] = (d.Date - d.CompetitionOpenSince).dt.days
    
    # cleanup int64/uint64 to save memory
    for col in d:
        if d[col].dtype in [np.int64, np.uint64]:
            if d[col].max() <= np.iinfo(np.int32).max:
                d[col] = d[col].astype(np.int32)
    
    return d

merge_right = [
    (store, 'Store'),
    (store_states, 'Store'),
    (state_names, 'State'),
    (weather, ['StateName', 'Date'], ['file', 'Date']),
    (expend_date(clean_trend, ['Year', 'Week']), ['State', 'Year', 'Week']),
]
fill_nan = [
    ('CompetitionOpenSinceMonth', 1, np.int32),
    ('CompetitionOpenSinceYear', 1900, np.int32),
    ('Promo2SinceWeek', 1, np.int32),
    ('Promo2SinceYear', 1900, np.int32),
    ('CompetitionDistance', 0, np.int32),
    ('trend',0, np.int32),
    ('CloudCover', -1, np.int32),
#     ('PromoInterval'),
#     ('Max_VisibilityKm'),
#     ('Mean_VisibilityKm'),
#     ('Min_VisibilitykM'),
#     ('Max_Gust_SpeedKm_h',),
#     ('Events',),
]
train_processed = preprocess(train, merge_right, drops=['file', 'Date_r', 'StateName'], nans=fill_nan)
#train_processed.head()

In [19]:
display(DataFrameSummary(train_processed).summary())

D:\dev\libs\Anaconda3\envs\tf140p35\lib\site-packages\pandas_summary\__init__.py:86: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  stats = pd.concat([counts, uniques, missing], axis=1)
D:\dev\libs\Anaconda3\envs\tf140p35\lib\site-packages\pandas_summary\__init__.py:63: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([self.df.describe(), self.columns_stats])[self.df.columns]


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,Week,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_PressurehPa,Mean_Sea_Level_PressurehPa,Min_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,trend,CompetitionOpenSince,CompetitionDaysOpen
count,1.01721e+06,1.01721e+06,NaN,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,NaN,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,NaN,NaN,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,NaN,NaN,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,992564,992564,992564,1.01721e+06,1.01721e+06,236167,1.01721e+06,1.01721e+06,NaN,1.01721e+06,1.01721e+06,NaN,1.01721e+06
mean,558.43,3.99834,NaN,5773.82,633.146,0.830107,0.381515,NaN,0.178647,2013.83,5.84676,15.7028,23.6155,NaN,NaN,5415.98,5.24476,1974.14,0.500564,12.1471,1955.94,NaN,NaN,14.3283,10.0739,5.90028,8.21675,5.835,3.26325,93.2838,73.9522,49.9234,1018.64,1015.43,1012.43,23.5428,12.1274,7.28499,22.6729,11.8708,48.4593,0.790271,5.0062,NaN,177.063,63.9962,NaN,14516.1
std,321.909,1.99739,NaN,3849.93,464.412,0.375539,0.485759,NaN,0.383056,0.777396,3.3261,8.78764,14.4334,NaN,NaN,7710.25,3.92852,50.8532,0.5,14.9479,55.8888,NaN,NaN,8.46478,7.23908,6.45973,6.14011,5.99337,6.03308,7.71733,13.2064,19.6264,7.91511,8.24871,8.74837,9.22237,4.87102,4.7714,8.94042,5.89719,13.1421,2.50262,2.38829,NaN,101.748,14.5175,NaN,18663.1
min,1,1,NaN,0,0,0,0,NaN,0,2013,1,1,1,NaN,NaN,0,1,1900,0,1,1900,NaN,NaN,-11,-13,-15,-14,-15,-73,44,30,4,976,974,970,0,0,0,3,2,21,0,-1,NaN,-1,0,NaN,-956
25%,280,2,NaN,3727,405,1,0,NaN,0,2013,3,8,11,NaN,NaN,700,1,1900,0,1,1900,NaN,NaN,8,4,1,3,1,-1,88,64,34,1014,1011,1007,11,10,3,16,8,39,0,4,NaN,79,57,NaN,889
50%,558,4,NaN,5744,609,1,0,NaN,0,2014,6,16,22,NaN,NaN,2320,4,2006,1,1,2009,NaN,NaN,15,10,6,8,6,3,94,75,49,1019,1016,1013,31,11,8,21,11,47,0,6,NaN,201,65,NaN,2771
75%,838,6,NaN,7856,837,1,1,NaN,0,2014,8,23,35,NaN,NaN,6880,9,2011,1,22,2012,NaN,NaN,21,16,11,13,11,8,100,84,65,1024,1021,1018,31,14,10,27,14,55,0.25,7,NaN,257,73,NaN,41455
max,1115,7,NaN,41551,7388,1,1,NaN,1,2015,12,31,52,NaN,NaN,75860,12,2015,1,50,2015,NaN,NaN,38,31,24,25,20,18,100,100,100,1043,1040,1038,31,31,31,101,53,111,58.93,8,NaN,360,100,NaN,42200
counts,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,509178,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,992564,992564,992564,1017209,1017209,236167,1017209,1017209,774961,1017209,1017209,1017209,1017209
uniques,1115,7,942,21734,4086,2,2,2,2,3,12,31,52,4,3,655,12,23,2,24,8,3,12,50,45,40,39,36,38,52,71,93,66,67,68,24,32,24,42,27,45,40,10,21,362,68,171,11345


In [20]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder, Imputer, StandardScaler

class DataMapper(object):
    def __init__(self, data, max_cat_unique=10, drop=None, categorical=None, numeric=None):
        self._data = data
        self.type_map = self._get_map(max_cat_unique, drop, categorical, numeric)
        self.cat_map, self.num_map = None, None

    def _fill_cat_nan(self, data):
        data_to_fill = []
        for col, col_type in self.type_map:
            if col_type == 'categorical' and np.any(data[col].isnull()):
                data_to_fill.append(col)
        if data_to_fill:
            data = data.copy()
            for col in data_to_fill:
                data.loc[data[col].isnull(), col] = ''  # fill empty for categories NaN
        return data
        
    def fit(self, data=None):
        if data is None:
            data = self._data
        data = self._fill_cat_nan(data)
        missing_numeric = []
        for col, col_type in self.type_map:
            if col_type != 'categorical' and data[col].isnull().sum() > 0:
                missing_numeric.append(col)
        
        if len(missing_numeric) > 0:
            raise Exception('[' + ', '.join(missing_numeric) + '] has missing values')
        
        cat_map = DataFrameMapper([(col, LabelEncoder()) for col, col_type in self.type_map if col_type == 'categorical'])
        num_map = DataFrameMapper([([col], StandardScaler()) for col, col_type in self.type_map if col_type == 'numeric'])
        # use [col] since StandardScaler().fit() require 2-D input
        self.cat_map = cat_map.fit(data)
        self.num_map = num_map.fit(data)

    def apply(self, data):
        data = self._fill_cat_nan(data)

        return self.cat_map.transform(data).astype(np.int64), self.num_map.transform(data).astype(np.float32)

    def _get_map(self, max_cat_unique=10, drop=None, categorical=None, numeric=None):
        null_info = []
        summary = DataFrameSummary(self._data)
        #type_maps = summary.columns_stats.copy().drop('missing_perc')
        type_maps = []
        
        for col in self._data:
            data_type = summary.columns_stats[col]['types']
#             if (drop and col in drop) or data_type not in [summary.TYPE_CATEGORICAL, summary.TYPE_BOOL,
#                                                            summary.TYPE_NUMERIC]:
#                 type_maps.drop(columns=[col], inplace=True)
#                 continue
            if drop and col in drop:
                continue
            map_type = ''
            if categorical and col in categorical:
                map_type = 'categorical'
            elif numeric and col in numeric:
                map_type = 'numeric'
            else:
                if data_type in [summary.TYPE_BOOL, summary.TYPE_CATEGORICAL]:
                    map_type = 'categorical'
                elif data_type == summary.TYPE_NUMERIC:
                    data_uniques = summary.columns_stats[col]['uniques']
                    if data_uniques <= max_cat_unique:
                        map_type = 'categorical'
                    else:
                        map_type = 'numeric'
            if summary.columns_stats[col]['missing'] > 0:
                if map_type == 'categorical':
                    null_info.append((col, ''))
                else:
                    null_info.append((col, 0))
            #type_maps[col]['types'] = map_type
            type_maps.append((col, map_type))
        return type_maps
mapper = DataMapper(train_processed, max_cat_unique=30,
                         categorical=['Store'],
                         drop=['Max_VisibilityKm', 'Mean_VisibilityKm', 'Min_VisibilitykM', 'Max_Gust_SpeedKm_h'])
mapper.fit()
#train_mapped = mapper.apply(train_processed)

D:\dev\libs\Anaconda3\envs\tf140p35\lib\site-packages\pandas_summary\__init__.py:86: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  stats = pd.concat([counts, uniques, missing], axis=1)
D:\dev\libs\Anaconda3\envs\tf140p35\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## Sample Data

In [21]:
# TODO: remove closed store

In [22]:
sample_num = 100000
np.random.seed(77)
idx = sorted(np.random.choice(train_processed.shape[0], sample_num, replace=False))

In [23]:
samples = train_processed.loc[idx].set_index('Date')

In [24]:
samples.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,Week,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_PressurehPa,Mean_Sea_Level_PressurehPa,Min_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,trend,CompetitionOpenSince,CompetitionDaysOpen
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-07-31,11,5,10457,1236,1,1,True,1,2015,7,31,31,a,c,960,11,2011,1,1,2012,"Jan,Apr,Jul,Oct",SH,18,14,8,11,7,4,88,61,33,1019,1017,1014,31.0,14.0,10.0,26,13,NaN,0.0,6,Rain,273,77,2011-11-15,1354
2015-07-31,22,5,6566,633,1,1,True,0,2015,7,31,31,a,a,1040,1,1900,1,22,2012,"Jan,Apr,Jul,Oct",BY,23,17,10,9,6,3,87,50,27,1019,1017,1014,NaN,NaN,NaN,19,6,NaN,0.0,-1,NaN,59,86,1900-01-15,42200
2015-07-31,44,5,6670,665,1,1,True,1,2015,7,31,31,a,a,540,6,2011,0,1,1900,NaN,BE,19,14,9,9,7,4,94,61,30,1019,1017,1014,10.0,10.0,10.0,23,16,NaN,0.0,6,NaN,282,74,2011-06-15,1507
2015-07-31,47,5,9379,1021,1,1,True,1,2015,7,31,31,a,c,270,4,2013,1,14,2013,"Jan,Apr,Jul,Oct",HH,17,13,8,10,8,4,100,70,35,1019,1017,1014,31.0,10.0,10.0,29,14,50.0,0.0,4,Rain,277,70,2013-04-15,837
2015-07-31,53,5,7540,690,1,1,True,1,2015,7,31,31,a,c,30360,9,2013,0,1,1900,NaN,SH,18,14,8,11,7,4,88,61,33,1019,1017,1014,31.0,14.0,10.0,26,13,NaN,0.0,6,Rain,273,77,2013-09-15,684


In [25]:
def seq_train_test_split(data, train_size=None, test_size=None):
    if train_size is None and test_size is None:
        raise RuntimeError()
    if train_size is None:
        if test_size < 1.0:
            train_size = 1.0 - test_size
        else:
            train_size = data.shape[0] - test_size
    assert train_size > 0
    if train_size < 1.0:
        train_size = int(data.shape[0] * train_size)
    else:
        train_size = int(train_size)
    return data[:train_size], data[train_size:]
sample_train, sample_valid = seq_train_test_split(samples, test_size=0.1)

In [26]:
len(sample_train), len(sample_valid)

(90000, 10000)

In [27]:
sp_cat_mapped_train, sp_num_mapped_train = mapper.apply(sample_train)
sp_cat_mapped_valid, sp_num_mapped_valid = mapper.apply(sample_valid)

D:\dev\libs\Anaconda3\envs\tf140p35\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [28]:
sp_y_orig_train, sp_y_orig_valid = sample_train.Sales, sample_valid.Sales

In [29]:
np.log(np.max(sp_y_orig_valid))

10.305714088793774

In [30]:
sp_y_train = np.clip(np.log(sp_y_orig_train + np.finfo(np.float32).eps) / np.log(np.max(sp_y_orig_train)), -1, None)
sp_y_valid = np.clip(np.log(sp_y_orig_valid + np.finfo(np.float32).eps) / np.log(np.max(sp_y_orig_valid)), -1, None)

In [31]:
def split_cols(data):
    return np.hsplit(data, data.shape[1])

# sp_mapped_train = torch.Tensor(split_cols(sp_cat_mapped_train) + split_cols(sp_num_mapped_train))

In [32]:
import torch
sp_cat_mapped_train, sp_num_mapped_train = torch.tensor(sp_cat_mapped_train, dtype=torch.long), torch.Tensor(sp_num_mapped_train)
sp_cat_mapped_valid, sp_num_mapped_valid = torch.tensor(sp_cat_mapped_valid, dtype=torch.long), torch.Tensor(sp_num_mapped_valid)

In [33]:
from torch import nn
class PaperModel(nn.Module):
    def __init__(self, mapper):
        super().__init__()
        self.cat_features = nn.ModuleList([self.get_emb(feat) for feat in mapper.cat_map.features])
        num_feature_size = len(mapper.num_map.features)*10
        self.num_features = nn.Sequential(nn.Linear(len(mapper.num_map.features), num_feature_size),
                                          nn.ReLU())
        self.main = nn.Sequential(#nn.Dropout(0.02),
                                  nn.Linear(sum([emb.embedding_dim for emb in self.cat_features]) + num_feature_size, 1000),
                                  nn.ReLU(),
                                  nn.Linear(1000, 500),
                                  nn.ReLU(),
                                  #nn.Dropout(0.2),
                                  nn.Linear(500, 1),
                                  nn.Sigmoid(),
                                 )
    
    def forward(self, cat_x, num_x):
        cat_features = [emb(cat_x[:, i]) for i, emb in enumerate(self.cat_features)]
        num_feature = self.num_features(num_x)
        input_feautures = torch.cat(cat_features + [num_feature], dim=1)
        
        return self.main(input_feautures)

    def get_emb(self, feature):
        name, feat = feature
        num_feature = len(feat.classes_)
        feature_out = min((num_feature + 1) // 2, 50)
        return nn.Embedding(num_feature, feature_out)

    def init_weight(self, m):
        class_name = m.__class__.__name__
        if class_name in ['Linear']:
            nn.init.uniform_(m.weight.data, -0.05, 0.05)
        elif class_name == 'Embedding':
            nn.init.uniform_(m.weight.data, -1, 1)
        elif class_name not in ['ModuleList', 'Sequential', self.__class__.__name__]:
            print('Not Init', class_name)

In [34]:
model = PaperModel(mapper)
model.apply(model.init_weight)
#r=model(sp_cat_mapped_train, sp_num_mapped_train)

Not Init ReLU
Not Init Dropout
Not Init ReLU
Not Init ReLU
Not Init Dropout
Not Init Sigmoid


PaperModel(
  (cat_features): ModuleList(
    (0): Embedding(1115, 50)
    (1): Embedding(7, 4)
    (2): Embedding(2, 1)
    (3): Embedding(2, 1)
    (4): Embedding(2, 1)
    (5): Embedding(2, 1)
    (6): Embedding(3, 2)
    (7): Embedding(12, 6)
    (8): Embedding(4, 2)
    (9): Embedding(3, 2)
    (10): Embedding(12, 6)
    (11): Embedding(23, 12)
    (12): Embedding(2, 1)
    (13): Embedding(24, 12)
    (14): Embedding(8, 4)
    (15): Embedding(4, 2)
    (16): Embedding(12, 6)
    (17): Embedding(27, 14)
    (18): Embedding(10, 5)
    (19): Embedding(22, 11)
  )
  (num_features): Sequential(
    (0): Linear(in_features=22, out_features=220, bias=True)
    (1): ReLU()
  )
  (main): Sequential(
    (0): Dropout(p=0.02)
    (1): Linear(in_features=363, out_features=1000, bias=True)
    (2): ReLU()
    (3): Linear(in_features=1000, out_features=500, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2)
    (6): Linear(in_features=500, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [36]:
from torch.utils.data import TensorDataset, DataLoader

dataloader = DataLoader(TensorDataset(sp_cat_mapped_train, sp_num_mapped_train, torch.tensor(sp_y_train)),
                        batch_size=128, shuffle=True, num_workers=os.cpu_count())
val_dataloader = DataLoader(TensorDataset(sp_cat_mapped_valid, sp_num_mapped_valid, torch.tensor(sp_y_valid)),
                            batch_size=128, shuffle=False, num_workers=os.cpu_count())

In [ ]:
from tqdm import tqdm_notebook as tqdm
import time

def train(model, opt, criterion, dataloader, epoches, valid_loader, start_epoch=0, device=torch.device('cuda')):
    model = model.to(device)
    for ep in range(start_epoch, epoches):
        bar = tqdm(total=len(dataloader) + len(valid_loader), desc='Ep {}'.format(ep + 1))
        train_loss_sum = 0
        val_loss_sum = 0
        last_time = time.time()
        for cat_x, num_x, y in dataloader:
            model.zero_grad()
            cat_x, num_x, y = cat_x.to(device), num_x.to(device), y.to(device)
            pred = model(cat_x, num_x)
            loss = criterion(pred.view(-1), y)
            loss.backward()
            opt.step()
            loss_val = float(loss.data)
            train_loss_sum += loss_val * len(y)
            if time.time() - last_time > 0.5:
                bar.set_postfix(loss=loss_val)
                last_time = time.time()
            bar.update()
        for cat_x, num_x, y in valid_loader:
            cat_x, num_x, y = cat_x.to(device), num_x.to(device), y.to(device)
            pred = model(cat_x, num_x)
            loss = criterion(pred.view(-1), y)
            val_loss_sum += float(loss.data) * len(y)
            bar.update()
        bar.set_postfix(loss=train_loss_sum/len(dataloader.dataset),
                        valid=val_loss_sum/len(valid_loader.dataset))
        bar.close()

for p in model.parameters():
    p.require_grad = True
train(model, torch.optim.Adam(model.parameters()), nn.L1Loss(), dataloader, epoches=5, valid_loader=val_dataloader)

In [269]:
len(dataloader.dataset)

90000

In [151]:
%pdb

Automatic pdb calling has been turned ON
